In [1]:
import math
import pandas as pd
import numpy as np

class Centrifugation(object):
    
    def __init__(self,MassIn,VolumeIn,Flowrate,ParallelUnits,SUHoldBags):
        self.MassIn = MassIn
        self.VolumeIn = VolumeIn
        self.Flowrate = Flowrate
        self.ParallelUnits = ParallelUnits
        self.SUHoldBags = SUHoldBags
        
        
    def Sizing(self):
        HarvestDuration=self.VolumeIn/(self.Flowrate*self.ParallelUnits)
        return HarvestDuration
    
    def MassBalance(self):
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        StepYield=Protocol.get_value('Step Yield','Centrifugation')
        MassOut=self.MassIn*StepYield
        VolumeOut=self.VolumeIn*Protocol.get_value('Dewatering Level','Centrifugation')
        ConcentrationOut=MassOut*1000/VolumeOut
        return MassOut,VolumeOut,ConcentrationOut
    
    def Timeframe(self):
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        #time to prepare for a batch
        PrepTime=Protocol.get_value('Preparation Time (hr)','Centrifugation')
        #time to operate a batch
        OperTime=self.VolumeIn/(self.Flowrate*self.ParallelUnits)
        #time to clean
        CIPTime=Protocol.get_value('CIP Time (hr)','Centrifugation')+Protocol.get_value('SIP Time (hr)','Centrifugation')
        #time to turnaround for next batch
        TurnAroundTime=Protocol.get_value('Turnaround Time (hr)','Centrifugation')
        #time to turnaround hold-tanks
        HoldTanksCIPandSIPTime=Protocol.get_value('CIP Time (hr)','Hold-Tanks')+Protocol.get_value('SIP Time (hr)','Hold-Tanks')
        return PrepTime,OperTime,CIPTime,TurnAroundTime,HoldTanksCIPandSIPTime
    
    def EquipmentAndMaterials(self):
        #Load Equipment from Database
        Equipment=pd.read_excel('Database.xlsx',sheetname='Equipment',index_col=0)
        #Load Materials from Database
        Materials=pd.read_excel('Database.xlsx',sheetname='Materials',index_col=0)
        #Load SOPs from Database
        Protocol=pd.read_excel('Database.xlsx',sheetname='SOPs',index_col=0)
        #Centrifuge, CIP, WFI, PW
        if self.Flowrate<=Equipment.get_value('Centrifuge (Small)','Base Size'):
            CentrifugeCost=Equipment.get_value('Centrifuge (Small)','Base Cost')*self.ParallelUnits
            CIPVolume=Equipment.get_value('Centrifuge (Small)','Base Size')*Protocol.get_value('CIP Time (hr)','Centrifugation')*self.ParallelUnits*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
            WFIVolume=Equipment.get_value('Centrifuge (Small)','Base Size')*Protocol.get_value('WFI Rinse (min)','Centrifugation')/60*self.ParallelUnits*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
            PWVolume=Equipment.get_value('Centrifuge (Small)','Base Size')*Protocol.get_value('PW Rinse (min)','Centrifugation')/60*self.ParallelUnits*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
        else:
            CentrifugeCost=Equipment.get_value('Centrifuge (Big)','Base Cost')*self.ParallelUnits
            CIPVolume=Equipment.get_value('Centrifuge (Big)','Base Size')*Protocol.get_value('CIP Time (hr)','Centrifugation')*self.ParallelUnits*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
            WFIVolume=Equipment.get_value('Centrifuge (Big)','Base Size')*Protocol.get_value('WFI Rinse (min)','Centrifugation')/60*self.ParallelUnits*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
            PWVolume=Equipment.get_value('Centrifuge (Big)','Base Size')*Protocol.get_value('PW Rinse (min)','Centrifugation')/60*self.ParallelUnits*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
        CentrifugeCIPCost=CIPVolume*Materials.get_value('CIP Buffer','Base Cost')
        CentrifugeWFICost=WFIVolume*Materials.get_value('WFI','Base Cost')
        CentrifugePWCost=PWVolume*Materials.get_value('PW','Base Cost')
        #Product hold-tank with GF, PW, WFI, CIP
        if Centrifugation.MassBalance(self)[1]/Protocol.get_value('Space Efficiency','Hold-Tanks')>Equipment.get_value('Hold-Tank','Max. Size'):
            NumberOfProductTanks=math.ceil((Centrifugation.MassBalance(self)[1]/Protocol.get_value('Space Efficiency','Hold-Tanks'))/Equipment.get_value('Hold-Tank','Max. Size'))
        else:
            NumberOfProductTanks=1
        ProductTankVolume=(Centrifugation.MassBalance(self)[1]/Protocol.get_value('Space Efficiency','Hold-Tanks'))/NumberOfProductTanks
        if ProductTankVolume<=10:
            CorrectedProductTankVolume=math.ceil(ProductTankVolume)
        elif ProductTankVolume<=100 and ProductTankVolume>10:
            CorrectedProductTankVolume=int(math.ceil(ProductTankVolume/10.0))*10
        elif ProductTankVolume<=1000 and ProductTankVolume>100:
            CorrectedProductTankVolume=int(math.ceil(ProductTankVolume/100.0))*100
        else:
            CorrectedProductTankVolume=int(math.ceil(ProductTankVolume/1000.0))*1000
        ProductHoldTankCost=NumberOfProductTanks*Equipment.get_value('Hold-Tank','Base Cost')*(CorrectedProductTankVolume/Equipment.get_value('Hold-Tank','Base Size'))**Equipment.get_value('Hold-Tank','Scaling Factor')
        ProductHoldTankGuard=NumberOfProductTanks*Materials.get_value('Hold-Tank Guard Filter','Base Cost')*(CorrectedProductTankVolume/Materials.get_value('Hold-Tank Guard Filter','Base Size'))**Materials.get_value('Hold-Tank Guard Filter','Scaling Factor')
        ProductCIPBuffer=NumberOfProductTanks*(Protocol.get_value('Acid Buffer','Hold-Tanks')+Protocol.get_value('Caustic Buffer','Hold-Tanks'))*CorrectedProductTankVolume*(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))
        ProductCIPBufferCost=ProductCIPBuffer*Materials.get_value('CIP Buffer','Base Cost')
        ProductHoldTankDiameter=NumberOfProductTanks*(CorrectedProductTankVolume*4000/(math.pi*Protocol.get_value('H/D tank ratio','Hold-Tanks')))**(1/3)
        ProductWFIVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*ProductHoldTankDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Hold-Tanks')/60*Protocol.get_value('WFI Rinse (min)','Hold-Tanks')
        ProductPWVolume=(1+Protocol.get_value('Buffer Overfill','Hold-Tanks'))*ProductHoldTankDiameter/100*Protocol.get_value('Water Flowrate (L/hr/m)','Hold-Tanks')/60*Protocol.get_value('PW Rinse (min)','Hold-Tanks')
        ProductWFIandPWCost=ProductWFIVolume*Materials.get_value('WFI','Base Cost')+ProductPWVolume*Materials.get_value('PW','Base Cost')
        TotalCIPVolume=CIPVolume+ProductCIPBuffer
        TotalCIPCost=CentrifugeCIPCost+ProductCIPBufferCost
        TotalWFIVolume=WFIVolume+ProductWFIVolume
        TotalWFIandPWCost=CentrifugeWFICost+CentrifugePWCost+ProductWFIandPWCost
        TotalPWVolume=PWVolume+ProductPWVolume
        return CentrifugeCost,ProductHoldTankCost,TotalCIPCost,TotalWFIandPWCost,ProductHoldTankGuard
    
    def CostBreakdown(self):
        EquipmentPurchaseCost=sum(Centrifugation.EquipmentAndMaterials(self)[0:2])
        #costs per batch
        ChemicalReagentsCost=sum(Centrifugation.EquipmentAndMaterials(self)[2:4])
        ConsumablesCost=sum(Centrifugation.EquipmentAndMaterials(self)[4:5])
        LabourCost=0
        return LabourCost,ChemicalReagentsCost,ConsumablesCost,EquipmentPurchaseCost

In [2]:
DiscStack=Centrifugation(Flowrate=1000,MassIn=10,ParallelUnits=1,SUHoldBags=False,VolumeIn=1000)
print(DiscStack.Sizing())
print(DiscStack.MassBalance())
print(DiscStack.Timeframe())
print(DiscStack.EquipmentAndMaterials())
print(DiscStack.CostBreakdown())

1.0
(8.5, 850.0, 10.0)
(1.0, 1.0, 3.0, 1.0, 6.0)
(510000, 68116.296619915607, 15180.0, 538.05527121746945, 457.82324613652179)
(0, 15718.055271217469, 457.82324613652179, 578116.29661991564)
